<a href="https://colab.research.google.com/github/praveentn/quoted/blob/master/quoted3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

os.chdir('/content/drive/My Drive/Quoted/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [23]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:

!ls

In [3]:
pwd

'/content/drive/My Drive/Quoted'

In [4]:
!git clone https://github.com/praveentn/quoted.git


Cloning into 'quoted'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.


In [5]:
!ls

quoted


In [6]:
cd quoted

/content/drive/My Drive/Quoted/quoted


In [7]:
!ls

author-quote.txt		       README.md
quote_generator_by_character_v1.ipynb  requirements.txt
quote_generator_by_character_v1.py


In [8]:
import pandas as pd 
import numpy as np 
import re
import sys

from nltk import word_tokenize
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.models import model_from_json
from keras.layers import Input, Activation, Dense, Dropout
from keras.layers import LSTM, Bidirectional

Using TensorFlow backend.


In [9]:
quotables = pd.read_csv('author-quote.txt', delimiter='\t', header=None)
quotables = quotables.rename(columns={0:'author', 1:'quote'})

In [10]:
quotables.sample(20)


,author,quote
3053,Arthur Darvill,I've started collecting taxidermy: I've got a ...
26736,Nora Ephron,What my mother believed about cooking is that ...
14721,Hugh Bonneville,"At 19, you know everything; by the time you're..."
2121,Andy Warhol,What's great about this country is that Americ...
14104,Henrik Ibsen,A forest bird never wants a cage.
30371,Ryan Eggold,You have to figure out 'who am I?' 'What do I ...
30639,Samuel Beckett,Words are all we have.
25795,Morena Baccarin,"My mom is an actress, but she never really pus..."
11408,Francis Bacon,"Hope is a good breakfast, but it is a bad supper."
15754,James Callis,"On a personal note, myself, I find religion - ..."


In [11]:
quotables['len_quotes'] = quotables.quote.map(lambda s: len(s))
quotes = list(quotables.quote + '\n')


removed_char = ['#', '$', '%', '(', ')', '=', ';' ,':',  '*', '+', '£' , '—','’']  
quotes_cleaned = []

for quote in quotes: 
    # remove unused character
    for s_char in removed_char:
        quote = quote.replace(s_char, ' ')
    
    # remove white space
    pattern = re.compile(r'\s{2,}')
    quote = re.sub(pattern, ' ', quote)

    quotes_cleaned.append(quote)

text = ' '.join(quotes_cleaned)
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [12]:
maxlen = 15
step = 6
sentences = []
next_chars = []

for quote in quotes_cleaned:
    for i in range(0, len(quote) - maxlen, step):
        sentences.append(quote[i: i + maxlen])
        next_chars.append(quote[i + maxlen])
    sentences.append(quote[-maxlen:])
    next_chars.append(quote[-1])
print('nb sequences:', len(sentences))

nb sequences: 753142


In [13]:
sentences[:20]


['If you live to ',
 ' live to be a h',
 'to be a hundred',
 'a hundred, I wa',
 'red, I want to ',
 ' want to live t',
 'to live to be a',
 'e to be a hundr',
 'e a hundred min',
 'ndred minus one',
 'minus one day s',
 'one day so I ne',
 'y so I never ha',
 ' never have to ',
 ' have to live w',
 'to live without',
 'e without you.\n',
 "Promise me you'",
 "e me you'll alw",
 "ou'll always re"]

In [14]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [15]:
x.shape, y.shape, len(chars)


((753142, 15, 73), (753142, 73), 73)

In [16]:
## Model 
print('Build model...')
input_sequences = Input((maxlen, len(chars)) , name="input_sequences")
lstm = Bidirectional(LSTM(256, return_sequences= True, input_shape=(maxlen, len(chars))), name = 'bidirectional')(input_sequences)
lstm = Dropout(0.1, name = 'dropout_bidirectional_lstm')(lstm)
lstm = LSTM(64, input_shape=(maxlen, len(chars)), name = 'lstm')(lstm)
lstm = Dropout(0.1,  name = 'drop_out_lstm')(lstm)

dense = Dense(15 * len(chars), name = 'first_dense')(lstm)
dense = Dropout(0.1,  name = 'drop_out_first_dense')(dense)
dense = Dense(5 * len(chars), name = 'second_dense')(dense)
dense = Dropout(0.1,  name = 'drop_out_second_dense')(dense)
dense = Dense(len(chars), name = 'last_dense')(dense)

next_char = Activation('softmax', name = 'activation')(dense)

model = Model([input_sequences], next_char)
model.compile(optimizer='adam', loss='categorical_crossentropy')

Build model...


In [17]:
model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_sequences (InputLayer) (None, 15, 73)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 15, 512)           675840    
_________________________________________________________________
dropout_bidirectional_lstm ( (None, 15, 512)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                147712    
_________________________________________________________________
drop_out_lstm (Dropout)      (None, 64)                0         
_________________________________________________________________
first_dense (Dense)          (None, 1095)              71175     
_________________________________________________________________
drop_out_first_dense (Dropou (None, 1095)              0   

In [18]:
model.fit([x], y,
         batch_size=128,
          epochs= 15
         )

Epoch 1/15
753142/753142 [==============================] - 788s 1ms/step - loss: 1.8728
Epoch 2/15
753142/753142 [==============================] - 799s 1ms/step - loss: 1.5307
Epoch 3/15
753142/753142 [==============================] - 765s 1ms/step - loss: 1.4260
Epoch 4/15
753142/753142 [==============================] - 755s 1ms/step - loss: 1.3672
Epoch 5/15
753142/753142 [==============================] - 778s 1ms/step - loss: 1.3280
Epoch 6/15
753142/753142 [==============================] - 758s 1ms/step - loss: 1.2979
Epoch 7/15
753142/753142 [==============================] - 783s 1ms/step - loss: 1.2741
Epoch 8/15
753142/753142 [==============================] - 761s 1ms/step - loss: 1.2538
Epoch 9/15
753142/753142 [==============================] - 751s 997us/step - loss: 1.2368
Epoch 10/15
753142/753142 [==============================] - 776s 1ms/step - loss: 1.2217
Epoch 11/15
753142/753142 [==============================] - 761s 1ms/step - loss: 1.2074
Epoch 12/15
75314

In [19]:
model.fit([x], y,
         batch_size=2048,
          epochs= 2
         )

Epoch 1/2
753142/753142 [==============================] - 89s 118us/step - loss: 1.0869
Epoch 2/2
753142/753142 [==============================] - 89s 118us/step - loss: 1.0656


In [20]:
model.fit([x], y,
         batch_size=1024,
          epochs= 2
         )

Epoch 1/2
753142/753142 [==============================] - 131s 174us/step - loss: 1.0661
Epoch 2/2
753142/753142 [==============================] - 131s 174us/step - loss: 1.0611


In [24]:

two_first_words = [bigram for bigram in [' '.join(word_tokenize(quote)[:2]) for quote in quotes] if len(bigram) <= maxlen]

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [25]:
def generate_quote(sentence = None, diversity = 0.8):
    
    if not sentence: ## if input is null then sample two first word from dataset
        random_index = np.random.randint(0, len(two_first_words))
        sentence = two_first_words[random_index]
        
    if len(sentence) > maxlen:
        sentence = sentence[-maxlen:]
    elif len(sentence) < maxlen:
        sentence = ' '*(maxlen - len(sentence)) + sentence
        
    generated = ''
    generated += sentence
    sys.stdout.write(generated)
    
    next_char = 'Empty'
    total_word = 0 
    
    max_word = 15
    
    while ((next_char not in ['\n', '.']) & (total_word <= 500)):
    
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        if next_char == ' ':
           total_word += 1
        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [26]:
generate_quote()


          I was just the terrorist, people will blame the film men me when I was going through the time for the now when you are anything new sort of fairly keeping that country is the most interesting.


In [27]:

# serialize model to JSON
model_json = model.to_json()
with open("model_char.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_char.h5")
print("Saved model to disk")


Saved model to disk
